### Variables

Molecular Data

In [ ]:
weight = 3.5 # in mg
molar = 851.5321 # in g/mol
usedpart = 1/5 # amount taken from initial 
vol = 50 # mL

Path

In [ ]:
path = r'C:\Users\jenso\PowerFolders\Forschung\UVVis\JK175\\'

TFA data

In [ ]:
tfa = 1 # mL
vTfa = 50 # mL
MTfa = 114.02 # g/mol
rTfa = 1.48 # g/mL
nTfa = tfa*rTfa/MTfa # mol
npVTfa = nTfa/vTfa # mol/mL
print(str(nTfa) + " mol, " + str(npVTfa) + " mol/mL")

Calculate Moles

In [ ]:
initial_moles = weight/1000/molar
moles = initial_moles*usedpart
print(str(initial_moles) + " mol initial, " + str(moles) + " mol used")

### Load Data

In [ ]:
import os
import fnmatch
from natsort import humansorted
import pandas as pd

In [ ]:
data = []
for root, dir, files in os.walk(path):
    for file in fnmatch.filter(files, "*.csv"):
        data.append(os.path.join(root, file))
data = humansorted(data)

In [ ]:
df = pd.DataFrame()
for item in data:
    df = pd.concat([df, (pd.read_csv(item, header=1, usecols=[0,1]))], axis=1)
df

### Print Uncorrected Spectra

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
num_spc = int(df.shape[1])
fig,ax = plt.subplots()
for i in range(0, num_spc, 2):
    x = df.iloc[:,i]
    y = df.iloc[:,i+1]
    plt.plot(x,y)
ax.set_xlim(250,1100)
ax.set_ylim(0,.75)

### Correction

In [ ]:
corr = df.copy(deep=True)
max = 0
idx = corr[corr.iloc[:,0].lt(300.0)].index[0]
soret = corr.iloc[:idx, 1].idxmax()


for i in range(0, num_spc, 2):
    fac = moles/(((i/2)+50)/1000)
    corr.iloc[:,i+1] = corr.iloc[:,i+1]/fac
    if max < corr.iloc[:,i+1][soret]:
        max = corr.iloc[:,i+1][soret]
corr

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
colors = [(0, 0, 0), (1, 0, 0)] # first color is black, last is red
cm = LinearSegmentedColormap.from_list(
        "Custom", colors, N=20)

In [ ]:
#cm = plt.get_cmap('Set3') 
cols = cm(np.linspace(0,1,int(num_spc/2) ))
fig,ax = plt.subplots()
for i in range(0, num_spc, 2):
    x = corr.iloc[:,i]
    y = corr.iloc[:,i+1]
    plt.plot(x,y, color=cols[int(i/2)])
ax.set_xlim(250,1100)
ax.set_ylim(0, max + 0.1*max)
